# Atmospheric pressure correction

This notebook uses example data to generate an atmospheric correction file.

In [ ]:
import stglib
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

basedir = '../examples/'

## Generate atmospheric .nc file

In [ ]:
def read_met_data(filename):
    a = pd.read_csv(filename, header=2, parse_dates=[2], index_col=2)

    # add 6 hours to get to UTC (but remian naive)
    a.index = a.index + pd.Timedelta('6 hours')
    a.index.rename('time', inplace=True)

    return xr.Dataset(a)

gndcrmet = read_met_data(basedir + 'GNDCRMET.csv') # This creates an xarray Dataset
gndcrmet = gndcrmet['BP'].to_dataset() # Let's keep only the BP variable
gndcrmet['BP'] = gndcrmet['BP']/100 # convert our atmos data (in millibars) to decibars
gndcrmet.to_netcdf(basedir + 'gndcrmet.nc') # This saves to a .nc file. Not required here as we will just be reading it back again

## Generate the atmpres.cdf file 
This generates the file and embeds the instrument-specific offset as an attr. The trickiest part of this process is determining what to use as an offset. After you run this cell, you will have your very own atmpres.cdf file!

In [ ]:
# Load the raw Aquadopp data
RAW = xr.load_dataset(basedir + '10761Aaqd-raw.cdf')

# Load the met data
gndcrmet = xr.load_dataset(basedir + 'gndcrmet.nc')

met = gndcrmet['BP'] # make a new met variable
met = met.rename('atmpres') # rename it to the standard atmpres variable name
met = met.reindex(time=RAW['time'], copy=True, method='nearest') # reindex the met data onto the Aquadopp time base
met.attrs.update(offset=-10.15) # set the atmospheric offset as an attribute
met.to_netcdf(basedir + 'atmpres.cdf') # save to disk

## Load clean data
Note that you need to run the proper run scripts with your generated atmpres.cdf files... this only uses example files.

Note also that the load_clean function below is set up to deal with older EPIC files. Files generated with modern versions of stglib will be in CF Conventions, but the process of generating an atmospheric pressure file is essentially the same.

In [ ]:
def load_clean(filename, basedir):
    fildir = basedir

    ds = xr.open_dataset(basedir + filename, decode_times=False, autoclose=True)
    ds['time'] = ds['time_cf']
    ds = ds.drop_vars('time2')
    
    return xr.decode_cf(ds)

VEL = load_clean('10761Aaqd-a.nc', basedir)

## View data
See how the raw and P_1ac data compare.

In [ ]:
plt.figure(figsize=(10,8))
RAW['Pressure'].plot()
VEL['P_1ac'].plot()
plt.show()